In [4]:

import numpy as np
import pandas as pd
import matplotlib.pyplot
from google.cloud import bigquery #For BigQu
pd.options.display.max_colwidth = 100

In [5]:
#
def estimate_gigabytes_scanned(query, bq_client):
    # see https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs#configuration.dryRun
    my_job_config = bigquery.job.QueryJobConfig()
    my_job_config.dry_run = True
    my_job = bq_client.query(query)

    return my_job

In [6]:
#Init
client = bigquery.Client()

hn_dataset_ref = client.dataset('nhtsa_traffic_fatalities', project='bigquery-public-data')
hn_dset = client.get_dataset(hn_dataset_ref)
hn_full = client.get_table(hn_dset.table('accident_2015'))

In [35]:
accidents_query_2015_all = """SELECT month_of_crash,
                                 day_of_week,
                                 hour_of_crash,
                                 manner_of_collision_name,
                                 light_condition_name,
                                 land_use_name,
                                 latitude,
                                 longitude,
                                 atmospheric_conditions_1_name,
                                 driver_maneuvered_to_avoid_name,
                                 number_of_drunk_drivers,
                                 number_of_fatalities
                          FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015` AS acc15
                          JOIN `bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2015` AS man15
                          ON acc15.consecutive_number = man15.consecutive_number
                          WHERE longitude < 0
                          AND longitude > -140
                      """ 

accidents_query_2016_all = """SELECT month_of_crash,
                                 day_of_week,
                                 hour_of_crash,
                                 manner_of_collision_name,
                                 light_condition_name,
                                 land_use_name,
                                 latitude,
                                 longitude,
                                 atmospheric_conditions_1_name,
                                 driver_maneuvered_to_avoid_name,
                                 number_of_drunk_drivers,
                                 number_of_fatalities
                          FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016` AS acc16
                          JOIN `bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016` as man16
                          ON acc16.consecutive_number = man16.consecutive_number
                          WHERE longitude < 0
                          AND longitude > -140
                      """ 


In [36]:
accidents_2015 = estimate_gigabytes_scanned(accidents_query_2015_all, client)
accidents_2016 = estimate_gigabytes_scanned(accidents_query_2016_all, client)

In [37]:
df15 = accidents_2015.to_dataframe()
df16 = accidents_2016.to_dataframe()
print(df16)
#
frames = [df15, df16]
results = pd.concat(frames)
# results = df15


       month_of_crash  day_of_week  hour_of_crash  \
0                  10            7             20   
1                  12            6              5   
2                  12            6              5   
3                  12            6              5   
4                   1            4             19   
5                   1            4             19   
6                   3            6              7   
7                  11            4             18   
8                   8            2              9   
9                   8            3             23   
10                  8            3             23   
11                 10            1             19   
12                 11            1             14   
13                 11            2              4   
14                 12            4             22   
15                  4            1              2   
16                  4            6              1   
17                  4            6            

In [54]:



ahh = {'\n': ' ', 'yyyyyyyy2':1, '23434?':1}
rep = results.replace(ahh)

reg = r"\(.*\\n
reg = "Not"
tom = results.replace({reg: ' '}, regex=True)
print(tom)

       month_of_crash  day_of_week  hour_of_crash  \
0                   2            4              7   
1                   2            1              2   
2                   4            4              6   
3                   4            5              5   
4                   1            1              8   
5                   2            1             21   
6                  10            2              8   
7                   9            1             22   
8                  10            2              8   
9                  12            1              6   
10                  2            2             12   
11                  9            2             14   
12                 11            3             12   
13                  3            4              9   
14                  1            3              7   
15                  2            7             11   
16                 12            3             13   
17                 12            1            

In [38]:
#hundy = results.sample(n=100000)
results.to_csv("hightway_fatals_manu.csv")